### Fig. 2.8, page 36
Also identical to Figure 1 from `model 2.1.xlsx` in worked examples.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_seir_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    model = CompartmentalModel(
        times=(0, config["end_time"]),
        compartments=compartments,
        infectious_compartments=("Infectious",),
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["population"] - config["infectious_seed"],
            "Infectious": config["infectious_seed"],
        }
    )
    
    # Transitions
    rec_rate = Parameter("rec_rate")
    model.add_infection_frequency_flow(
        name="infection",
        contact_rate=Parameter("r0") * rec_rate,
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=Parameter("infous_rate"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=rec_rate,
        source="Infectious", 
        dest="Immune"
    )
    
    # Output
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression",
        raw_results=True,
    )
    return model

In [ ]:
model_config = {
    "population": 1e5,
    "infectious_seed": 1.,
    "end_time": 200.,
}

parameters = {
    "r0": 2.,
    "infous_rate": 1. / 2.,
    "rec_rate": 1. / 2.,
}

In [ ]:
seir_model = build_seir_model(model_config)
seir_model.run(parameters=parameters)
compartments = seir_model.get_outputs_df()

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
twin_ax = left_ax.twinx()

left_ax.plot(compartments["Susceptible"], color="0.2", label="Susceptible")
left_ax.plot(compartments["Immune"], color="0.8", label="Immune")
twin_ax.plot(seir_model.times, seir_model.derived_outputs["incidence"], color="0.5", label="Infectious")

left_ax.legend()
left_ax.set_xlabel("Time (days)")
left_ax.set_xlim(0, 200)
left_ax.set_xticks(range(0, 250, 50))
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_ylim(0, 1e5)
left_ax.set_ylabel("Number of susceptible or immune individuals")
left_ax.spines.top.set_visible(False)

twin_ax.legend(loc=4)
twin_ax.set_ylim(0, 5000)
twin_ax.set_ylabel("Number of new infectious persons/day")
twin_ax.spines.top.set_visible(False)

image = plt.imread("./figures/fig_2_8.jpg")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");